# ETL

ETL is comprised of 3 steps:
- Extract: Open the Excel files, read the data into Python memory
- Transform: Convert the data to the format I will use in my database
- Load: Put the data in my local SQL database

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option('display.max_rows', 100)

## Extract

In [2]:
def collect_name_and_excel_paths(num_files= 0):
    count = 0
    base_path = Path("chargemaster-cdm-2021")
    hospitals = sorted([path for path in base_path.iterdir() if not path.name.startswith('.')])
    hospital_sheets = []
    allowed_extensions = set([".xls", ".xlsx", ".csv"])
    
    for hospital in hospitals:
        # Skip hospitals with multiple excel files for now. 
        # TODO: Implement handling multiple excels and validate data from all hospitals.
        files = [(hospital.name, file) for file in hospital.iterdir() if file.suffix in allowed_extensions and not file.name.startswith("~")]
        if len(files) == 1:
            hospital_sheets.extend(files)
            count+=1
        if num_files >= 1 and count >= num_files:
            break

    return hospital_sheets

names_and_files = collect_name_and_excel_paths()
names_and_files[:5]

[('AHMC Anaheim Regional Medical Center',
  PosixPath('chargemaster-cdm-2021/AHMC Anaheim Regional Medical Center/106301098_CDM_All_2021.xlsx')),
 ('AHMC Seton Medical Center',
  PosixPath('chargemaster-cdm-2021/AHMC Seton Medical Center/106410817_CDM_All_2021.xlsx')),
 ('Alameda Hospital',
  PosixPath('chargemaster-cdm-2021/Alameda Hospital/106010735_CDM_All_2021.xlsx')),
 ('Alta Bates Summit Medical Center',
  PosixPath('chargemaster-cdm-2021/Alta Bates Summit Medical Center/106010937_CDM_All_2021.xlsx')),
 ('Alta Bates-Summit Medical Center - Alta Bates Campus',
  PosixPath('chargemaster-cdm-2021/Alta Bates-Summit Medical Center - Alta Bates Campus/106010739_CDM_All_2021.xlsx'))]

In [3]:
def data_from_excel(filepath):
    # Handle all extensions, and all sheet names. 
    if filepath.suffix not in ['.xlsx', '.xls']:
        raise Exception("file extension not supported")
        
    sheet_name = "AB 1045 Form"
    sheet_names = pd.ExcelFile(filepath).sheet_names
    for name in sheet_names:
        if "1045" in name:
            sheet_name = name
    
    item = pd.read_excel(filepath, sheet_name=sheet_name)
    return item

name, path = names_and_files[16]
data = data_from_excel(path)
data.head(100)

,Hospital Name: Ballard Rehabilitation Hospital,Unnamed: 1,Unnamed: 2
0,OSHPD Facility No: 106364121,NaN,NaN
1,"Effective Date of Charges: June 1, 2021",NaN,NaN
2,NaN,NaN,NaN
3,In response to requests from hospitals and the...,NaN,NaN
4,Evaluation & Management Services (CPT Codes 99...,2020 CPT Code,Average Charge
5,"Emergency Room Visit, Level 2 (low to moderate...",99282,NaN
6,"Emergency Room Visit, Level 3 (moderate severity)",99283,NaN
7,"Emergency Room Visit, Level 4 (high severity w...",99284,NaN
8,"Emergency Room Visit, Level 4 (high severity w...",99285,NaN
9,"Outpatient Visit, established patient, 15 minutes",99213,NaN


## Transform

For now, we limit to extracting the rows with numeric CPT and charge values. In the future we can extract the number of reported procedures to validate out data extraction process. We can also extract the hospital ID and the date of the report. 

In [4]:
def transform_data(data, hospital_name):
    mask = data.astype('str').iloc[:, 2].str.contains("\d+")
    # for now, ignore procedures without a fully numeric CPT field
    # TODO: implement validation. Collected procedures count in file must be same as extracted. 
    mask2 = data.astype('str').iloc[:, 1].str.contains("^\d+$")
    reported_procedures = data[(mask & mask2)]
    idx = pd.Index(range(len(reported_procedures)))
    reported_procedures.set_index(idx, inplace=True)

    reported_procedures.columns = ["Procedure", "CPT Code", "Average Charge"]

    hospital_column = [hospital_name] * len(reported_procedures)
    hospital_series = pd.Series(np.array(hospital_column))
    reported_procedures["Hospital"] = hospital_series

    reported_procedures = reported_procedures[["Hospital", "CPT Code", "Procedure", "Average Charge"]]
    reported_procedures.set_index("Hospital", inplace=True)

    reported_procedures["CPT Code"] = reported_procedures["CPT Code"].apply(np.int64)
    # Convert charges to cents and round. 
    reported_procedures["Average Charge"] = (reported_procedures["Average Charge"] * 100).astype(np.int64)

    return reported_procedures

reported_procedures = transform_data(data, name)
reported_procedures

/var/folders/yv/3zzm4_cs25v74y31mq6j8_mh0000gr/T/ipykernel_60607/3297213217.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reported_procedures["Hospital"] = hospital_series


,CPT Code,Procedure,Average Charge
Hospital,,,
Ballard Rehabilitation Hospital,80048,Basic Metabolic Panel,1163
Ballard Rehabilitation Hospital,85027,"Complete Blood Count, automated",889
Ballard Rehabilitation Hospital,85025,"Complete Blood Count, with differential WBC, a...",1069
Ballard Rehabilitation Hospital,80053,Comprehensive Metabolic Panel,1453
Ballard Rehabilitation Hospital,82550,"Creatine Kinase (CK), (CPK), Total",894
Ballard Rehabilitation Hospital,80061,Lipid Panel,1841
Ballard Rehabilitation Hospital,85730,Partial Thromboplastin Time,825
Ballard Rehabilitation Hospital,85610,Prothrombin Time,540
Ballard Rehabilitation Hospital,84443,Thyroid Stimulating Hormone,2310


## Load

Convert to a database readable format and load into a database. 

In [5]:
def write_data(reported_procedures, dst_folder, hospital_name):
    output_dir = Path(dst_folder)
    output_dir.mkdir(parents=True, exist_ok=True)
    output_path = output_dir / f"{hospital_name}_reported_procedures.csv"
    print(f"Writing reported procedures to {output_path}")
    reported_procedures.to_csv(output_path)
    
write_data(reported_procedures, "reported_procedures", name)

Writing reported procedures to reported_procedures/Ballard Rehabilitation Hospital_reported_procedures.csv


## Execute

In [16]:
NUM_FILES = 400
DST_FOLDER = "reported_procedures"

names_and_files = collect_name_and_excel_paths(NUM_FILES)
for name, file in names_and_files:
    print("Parsing ", file)
    try: 
        data = data_from_excel(file)
    except:
        print(f"Failed to read data from hospital {name}")
        continue
        
    try: 
        reported_procedures = transform_data(data, name)
    except:
        print(f"Failed to transform data from hospital {name}")
        continue
        
    if len(reported_procedures) == 0:
        print(f"Hospital {name} seems to contain no reported procedures")
        display(reported_procedures)
        continue
    write_data(reported_procedures, DST_FOLDER, name)

Parsing  chargemaster-cdm-2021/AHMC Anaheim Regional Medical Center/106301098_CDM_All_2021.xlsx


/var/folders/yv/3zzm4_cs25v74y31mq6j8_mh0000gr/T/ipykernel_60607/3297213217.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reported_procedures["Hospital"] = hospital_series
/var/folders/yv/3zzm4_cs25v74y31mq6j8_mh0000gr/T/ipykernel_60607/3297213217.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reported_procedures["CPT Code"] = reported_procedures["CPT Code"].apply(np.int64)
/var/folders/yv/3zzm4_cs25v74y31mq6j8_mh0000gr/T/ipykernel_60607/3297213217.py:21: SettingWithCopyWarning: 
A value is try

Writing reported procedures to reported_procedures/AHMC Anaheim Regional Medical Center_reported_procedures.csv
Parsing  chargemaster-cdm-2021/AHMC Seton Medical Center/106410817_CDM_All_2021.xlsx


/var/folders/yv/3zzm4_cs25v74y31mq6j8_mh0000gr/T/ipykernel_60607/3297213217.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reported_procedures["Hospital"] = hospital_series
/var/folders/yv/3zzm4_cs25v74y31mq6j8_mh0000gr/T/ipykernel_60607/3297213217.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reported_procedures["CPT Code"] = reported_procedures["CPT Code"].apply(np.int64)
/var/folders/yv/3zzm4_cs25v74y31mq6j8_mh0000gr/T/ipykernel_60607/3297213217.py:21: SettingWithCopyWarning: 
A value is try

Writing reported procedures to reported_procedures/AHMC Seton Medical Center_reported_procedures.csv
Parsing  chargemaster-cdm-2021/Alameda Hospital/106010735_CDM_All_2021.xlsx


...

/var/folders/yv/3zzm4_cs25v74y31mq6j8_mh0000gr/T/ipykernel_60607/3297213217.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reported_procedures["Hospital"] = hospital_series
/var/folders/yv/3zzm4_cs25v74y31mq6j8_mh0000gr/T/ipykernel_60607/3297213217.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reported_procedures["CPT Code"] = reported_procedures["CPT Code"].apply(np.int64)
/var/folders/yv/3zzm4_cs25v74y31mq6j8_mh0000gr/T/ipykernel_60607/3297213217.py:21: SettingWithCopyWarning: 
A value is try

Writing reported procedures to reported_procedures/Whittier Hospital Medical Center_reported_procedures.csv
Parsing  chargemaster-cdm-2021/Woodland Memorial Hospital/106571086_CDM_All_2021.xlsx


/var/folders/yv/3zzm4_cs25v74y31mq6j8_mh0000gr/T/ipykernel_60607/3297213217.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reported_procedures["Hospital"] = hospital_series
/var/folders/yv/3zzm4_cs25v74y31mq6j8_mh0000gr/T/ipykernel_60607/3297213217.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reported_procedures["CPT Code"] = reported_procedures["CPT Code"].apply(np.int64)
/var/folders/yv/3zzm4_cs25v74y31mq6j8_mh0000gr/T/ipykernel_60607/3297213217.py:21: SettingWithCopyWarning: 
A value is try

Writing reported procedures to reported_procedures/Woodland Memorial Hospital_reported_procedures.csv
Parsing  chargemaster-cdm-2021/Zuckerberg San Francisco General Hospital & Trauma Center/106380939_CDM_All_2021.xlsx
Writing reported procedures to reported_procedures/Zuckerberg San Francisco General Hospital & Trauma Center_reported_procedures.csv


/var/folders/yv/3zzm4_cs25v74y31mq6j8_mh0000gr/T/ipykernel_60607/3297213217.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reported_procedures["Hospital"] = hospital_series
/var/folders/yv/3zzm4_cs25v74y31mq6j8_mh0000gr/T/ipykernel_60607/3297213217.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reported_procedures["CPT Code"] = reported_procedures["CPT Code"].apply(np.int64)
/var/folders/yv/3zzm4_cs25v74y31mq6j8_mh0000gr/T/ipykernel_60607/3297213217.py:21: SettingWithCopyWarning: 
A value is try

In [15]:
NUM_FILES = 400
DST_FOLDER = "reported_procedures"

names_and_files = [("GLENDORA", Path("chargemaster-cdm-2021/Glendora Oaks Behavioral Health Hospital/106190328_CDM_All_2021.xlsx"))]

for name, file in names_and_files:
    print("Parsing ", file)
    try: 
        data = data_from_excel(file)
    except:
        print(f"Failed to read data from hospital {name}")
        continue
    try: 
        reported_procedures = transform_data(data, name)
    except:
        print(f"Failed to transform data from hospital {name}")
        continue
    reported_procedures = transform_data(data, name)
    if len(reported_procedures) == 0:
        print(f"Hospital {name} seems to contain no reported procedures")
        display(reported_procedures)
        continue
    write_data(reported_procedures, DST_FOLDER, name)

Parsing  chargemaster-cdm-2021/Glendora Oaks Behavioral Health Hospital/106190328_CDM_All_2021.xlsx
Failed to transform data from hospital GLENDORA
